### 获取斑块的面积
为了能够自动获取标注中很小的斑块, 自动统计并计算斑块的面积

In [177]:
import os
import json
import labelme
import cv2
import numpy as np
import PIL.Image
import PIL.ImageDraw


def shape_to_mask(img_size:tuple, points, line_width=1, point_size=1):
    """
        根据多边形的点在空白图中画出多边形, 填充像素为127的灰度. 返回一个bool值的矩阵, 矩阵中处多边形覆盖区域外全部为False
        input:
            img_size: original image size(width*heigth)
            points: all points of the polygon
        return: image mask (ndarray)
    """
    img_mask = np.zeros(img_size[:2], dtype=np.uint8)
    img_mask = PIL.Image.fromarray(img_mask)
    draw = PIL.ImageDraw.Draw(img_mask)
    xy = [tuple(point) for point in points]
    if len(xy) < 3:
        errmsg = 'Polygon must have points more than 2'
        raise ValueError(errmsg)
    draw.polygon(xy=xy, outline=128, fill=128)
    img_mask = np.array(img_mask, dtype=bool)
    return img_mask


def get_polygon_area(polygon_array):
    """
        根据polygon的mask得到这个polygon的像素数量, 用来代表面积
        input:
            polygon_array: 在空白画布上画出polygon后转化为bool的矩阵
        return:
            (pixel_count, pixel_percentage)
    """
    if polygon_array.dtype != np.bool:
        errmsg = 'Polygon array must in bool dtype'
        raise ValueError(errmsg)
    pixel_count = np.count_nonzero(polygon_array)
    pixel_percentage = pixel_count / polygon_array.size
    return (pixel_count, pixel_percentage)


def get_polygon_pixel_mean_value(img_array, polygon_array):
    """
        根据polygon的mask获取polygon的所有像素的灰度值的求和与均值
        input:
            img_array: 原始的图片的灰度值的矩阵 shape=(width, heigth)
            polygon_array: polygon的mask的矩阵, 值为bool.
        return:
            (polygon_pixel_mean, polygon_pixel_sum): polygon覆盖区域像素灰度值的求和与均值
    """
    if polygon_array.dtype != np.bool:
        errmsg = 'Polygon array must in bool dtype'
        raise ValueError(errmsg)
    if img_array.dtype != np.uint8:
        errmsg = 'image array must in uint8 dtype'
        raise ValueError(errmsg)
    
    polygon_img_array = img_array.copy()
    polygon_img_array[np.logical_not(polygon_array)] = 0

    polygon_pixel_sum = np.sum(polygon_img_array)
    #polygon_pixel_mean = polygon_pixel_sum / np.count_nonzero(polygon_img_array)
    polygon_pixel_mean = (polygon_pixel_sum ) / (np.count_nonzero(polygon_img_array) + 1)

    return (polygon_pixel_mean, polygon_pixel_sum)


def crop_sub_mask(polygon_origin, polygon_sub):
    """
        将origin的mask中剪掉sub的mask. 适用于血管中有大斑块的情况, 这样会导致血管的像素灰度值均值受到影响. 
        要求两个矩阵shape相等
        input:
            polygon_origin: 大的polygon的mask的矩阵, 原始的矩阵
            polygon_sub: 需要被剪掉的部分的矩阵
        return:
            polygon_new: 将polygon_origin中polygon_sub为True的部分改为False的矩阵
    """
    for array in (polygon_origin, polygon_sub):
        if array.dtype != np.bool:
            errmsg = 'Polygon array must in bool dtype'
            raise ValueError(errmsg)
            
    polygon_new = polygon_origin.copy()
    polygon_new[polygon_sub] = False

    return polygon_new


In [178]:
import os
import json
import labelme
import cv2

root_dir = r"F:\workspace\for_demos\07_Plaque_13_YY_02_frms"
label_dict = {
    '_background_': 0,
    'Plaque': 1,
    'CA': 2,
    'JV': 3,
}
img_file = os.path.join(root_dir, "frm-0001.png")
json_file = img_file.split('.png')[0]+".json"

img_data = cv2.imread(img_file, cv2.IMREAD_GRAYSCALE)
with open(json_file, 'r') as f:
    json_data = json.load(f)

print(img_file, json_file, json_data['shapes'])

F:\workspace\for_demos\07_Plaque_13_YY_02_frms\frm-0001.png F:\workspace\for_demos\07_Plaque_13_YY_02_frms\frm-0001.json [{'label': 'CA', 'line_color': None, 'fill_color': None, 'points': [[753.0, 307.0], [819.0, 291.0], [877.0, 295.0], [945.0, 314.0], [976.0, 382.0], [976.0, 443.0], [953.0, 482.0], [913.0, 498.0], [791.0, 518.0], [691.0, 488.0], [674.0, 398.0], [704.0, 333.0]]}, {'label': 'JV', 'line_color': None, 'fill_color': None, 'points': [[1027, 404], [1026, 422], [1045, 423], [1049, 404]]}, {'label': '不确定区域:UncertainZone', 'line_color': None, 'fill_color': None, 'points': [[377.0, 0.0], [392.0, 1004.0], [1339, 498]]}, {'label': 'Plaque', 'line_color': None, 'fill_color': None, 'points': [[772.0, 414.0], [806.0, 426.0], [852.0, 453.0], [898.0, 458.0], [927.0, 467.0], [862.0, 478.0], [788.0, 477.0], [772.0, 452.0]]}]


In [179]:
print(json_data['shapes'][0], json_data['shapes'][2])
msk0 = shape_to_mask(img_data.shape, json_data['shapes'][0]['points'])
msk1 = shape_to_mask(img_data.shape, json_data['shapes'][2]['points'])
msk = crop_sub_mask(msk0, msk1)
pixels = get_polygon_area(msk1)
grays = get_polygon_pixel_mean_value(img_data, msk0)
print("0:", pixels, grays)
pixels = get_polygon_area(msk)
grays = get_polygon_pixel_mean_value(img_data, msk)
print("delta:", pixels, grays)

{'label': 'CA', 'line_color': None, 'fill_color': None, 'points': [[753.0, 307.0], [819.0, 291.0], [877.0, 295.0], [945.0, 314.0], [976.0, 382.0], [976.0, 443.0], [953.0, 482.0], [913.0, 498.0], [791.0, 518.0], [691.0, 488.0], [674.0, 398.0], [704.0, 333.0]]} {'label': '不确定区域:UncertainZone', 'line_color': None, 'fill_color': None, 'points': [[377.0, 0.0], [392.0, 1004.0], [1339, 498]]}
0: (480192, 0.279368507279294) (37.40620957309185, 809620)
delta: (0, 0.0) (0.0, 0)


In [127]:
import math
from PIL import ImageFont
import PIL.ImageFilter, PIL.Image, PIL.ImageDraw
import numpy as np
import PIL.Image
import PIL.ImageDraw
import random

def rndChar():
    return chr(random.randint(65, 90))

def rndColor2():
    return (random.randint(64, 255), random.randint(64, 255), random.randint(64, 255))

width = 60 * 4
height = 60
word_size = 50 #文字大小
word_css  = r"C:\Windows\Fonts\SIMYOU.TTF"

def shape_to_mask(img_shape, points, shape_type=None,
                  line_width=10, point_size=5):
    mask = np.zeros(img_shape[:2], dtype=np.uint8)
    mask = PIL.Image.fromarray(mask)
    draw = PIL.ImageDraw.Draw(mask)
    font = ImageFont.truetype(word_css,word_size)
    xy = [tuple(point) for point in points]
    assert len(xy) > 2, 'Polygon must have points more than 2'
    print(f"points: {xy}")
    draw.polygon(xy=xy, outline=128, fill=1)
    #for t in range(4):
    #    draw.text((60 * t + 10, 10), rndChar(), fill=rndColor2())
    #draw.line((0, 0) + (100, 100), fill=128)
    #draw.line((0, img_shape[1], img_shape[0], 0), fill=128)
    mask = np.array(mask, dtype=bool)
    #mask.show()
    #mask = np.array(mask, dtype=np.uint8)
    return mask

def shapes_to_label(img_shape, shapes, label_name_to_value, type='class'):
    assert type in ['class', 'instance']

    cls = np.zeros(img_shape[:2], dtype=np.int32)
    if type == 'instance':
        ins = np.zeros(img_shape[:2], dtype=np.int32)
        instance_names = ['_background_']
    for shape in shapes:
        points = shape['points']
        label = shape['label']
        
        shape_type = shape.get('shape_type', None)
        if type == 'class':
            cls_name = label
        elif type == 'instance':
            cls_name = label.split('-')[0]
            if label not in instance_names:
                instance_names.append(label)
            ins_id = instance_names.index(label)
        cls_id = label_name_to_value[cls_name]
        print(f"label:{label}, idx={cls_id}")
        mask = shape_to_mask(img_shape[:2], points, shape_type)
        cls[mask] = cls_id
        print(f"cls_id={cls_id}, count={np.count_nonzero(cls==cls_id)}")
        if type == 'instance':
            ins[mask] = ins_id

    if type == 'instance':
        return cls, ins
    return cls

In [128]:
help(labelme.utils.shapes_to_label)
help(labelme.utils.shape_to_mask)

Help on function shapes_to_label in module labelme.utils.shape:

shapes_to_label(img_shape, shapes, label_name_to_value, type='class')

Help on function shape_to_mask in module labelme.utils.shape:

shape_to_mask(img_shape, points, shape_type=None, line_width=10, point_size=5)



In [139]:

#polygons_masks = labelme.utils.shapes_to_label(img_data.shape, json_data['shapes'], label_dict)
polygons_masks = shapes_to_label(img_data.shape, json_data['shapes'], label_dict)
#cv2.imshow("img", (polygons_masks*128))
#cv2.waitKey(0)
#cv2.destroyAllWindows()

print(f"image data mean:{np.sum(img_data)/np.count_nonzero(img_data)}")

mask_plaque_bool = polygons_masks==1
print(type(mask_plaque_bool), mask_plaque_bool.shape,mask_plaque_bool[0][0])
img_data[np.logical_not(mask_plaque_bool)]=0
plaque_mean = np.sum(img_data)/np.count_nonzero(img_data)
print(f"plaque_mean:{plaque_mean}")

label:CA, idx=2
points: [(753.0, 307.0), (819.0, 291.0), (877.0, 295.0), (945.0, 314.0), (976.0, 382.0), (976.0, 443.0), (953.0, 482.0), (913.0, 498.0), (791.0, 518.0), (691.0, 488.0), (674.0, 398.0), (704.0, 333.0)]
cls_id=2, count=55587
label:Plaque, idx=1
points: [(772.0, 414.0), (806.0, 426.0), (852.0, 453.0), (898.0, 458.0), (927.0, 467.0), (862.0, 478.0), (788.0, 477.0), (772.0, 452.0)]
cls_id=1, count=4795
image data mean:79.13080260303687
<class 'numpy.ndarray'> (1004, 1712) False
plaque_mean:79.13080260303687


In [119]:
for shape in json_data['shapes']:
    points = shape['points']
    #mask = labelme.utils.shape_to_mask(img_data.shape[:2], points, None)
    mask = shape_to_mask(img_data.shape[:2], points, None)
    print(type(mask), mask.shape)
    print(f"={cls_id}, count={np.count_nonzero(cls==cls_id)}")
    
    cv2.imshow("img", mask)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

points: [(772.0, 414.0), (806.0, 426.0), (852.0, 453.0), (898.0, 458.0), (927.0, 467.0), (862.0, 478.0), (788.0, 477.0), (772.0, 452.0)]
<class 'numpy.ndarray'> (1004, 1712)


NameError: name 'cls_id' is not defined

label:Plaque, idx=1
points: [(772.0, 414.0), (806.0, 426.0), (852.0, 453.0), (898.0, 458.0), (927.0, 467.0), (862.0, 478.0), (788.0, 477.0), (772.0, 452.0)]
cls_id=1, count=5900
label:CA, idx=2
points: [(753.0, 307.0), (819.0, 291.0), (877.0, 295.0), (945.0, 314.0), (976.0, 382.0), (976.0, 443.0), (953.0, 482.0), (913.0, 498.0), (791.0, 518.0), (691.0, 488.0), (674.0, 398.0), (704.0, 333.0)]
cls_id=2, count=56488


In [100]:
type(polygons_masks)

numpy.ndarray

In [101]:
print(polygons_masks.shape)
plaque = np.argwhere(polygons_masks==1)
ca = np.argwhere(polygons_masks==2)
print(np.count_nonzero(polygons_masks==1))
print(np.count_nonzero(polygons_masks==2))

(1004, 1712)
0
5136
